## Board

## Sample Game

In [30]:
"""This file contains a collection of player classes for comparison with your
own agent and example heuristic functions.
"""

from random import randint

if __name__ == "__main__":
    
    # from isolation.py get the Board class
    from isolation import Board

    ## Setup
    # create an isolation board (by default 7x7) and some players
    player1 = RandomPlayer()
    player2 = GreedyPlayer()
    game = Board(player1, player2)

    # place player 1 on the board at row 2, column 3, then place player 2 on
    # the board at row 0, column 5; display the resulting board state.  Note
    # that .apply_move() changes the calling object
    
    # apply move and print the game board
    game.apply_move((2, 3))
    game.apply_move((0, 5))
    print(game.to_string())
    
    ## Play
    
    # players take turns moving on the board, so player1 should be next to move
    assert(player1 == game.active_player)

    # get a list of the legal moves available to the active player
    print(game.get_legal_moves())

    # get a successor of the current state by making a copy of the board and
    # applying a move. Notice that this does NOT change the calling object
    # (unlike .apply_move()).
    
    # make a new board as a branch of the old board, and assert not the same as before, then print both 
    new_game = game.forecast_move((1, 1))
    assert(new_game.to_string() != game.to_string())
    print("\nOld state:\n{}".format(game.to_string()))
    print("\nNew state:\n{}".format(new_game.to_string()))

    # play the remainder of the game automatically -- outcome can be "illegal
    # move" or "timeout"; it should _always_ be "illegal move" in this example
    
    # play the game, and get the winner, history, and outcome from 
    winner, history, outcome = game.play()
    print("\nWinner: {}\nOutcome: {}".format(winner, outcome))
    print(game.to_string())
    print("Move history:\n{!s}".format(history))

### Scoring (Evaluation Functions)

In [34]:
def null_score(game, player):
    """
    This heuristic presumes no knowledge for non-terminal states, and
    returns the same uninformative value for all other states.
    
    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).
    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).
    Returns
    ----------
    float
        The heuristic value of the current game state.
    """
    # if no one has lost or one yet
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")
    
    # this eval fn returns 0
    return 0.


def open_move_score(game, player):
    """
    The basic evaluation function described in lecture that outputs a score
    equal to the number of moves open for your computer player on the board.
    (#my_moves)
    
    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).
    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).
        
    Returns
    ----------
    float
        The heuristic value of the current game state
    """
    
    # if this player has already lost, your eval fn is -inf
    if game.is_loser(player):
        return float("-inf")

    # if this player has already won, your eval fn is inf
    if game.is_winner(player):
        return float("inf")

    # if no one has won yet
    return float(len(game.get_legal_moves(player)))


def improved_score(game, player):
    """
    The "Improved" evaluation function discussed in lecture that outputs a
    score equal to the difference in the number of moves available to the
    two players.
    Parameters
    (#my_moves - #their_moves)
    
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).
        
    player : hashable
        One of the objects registered by the game object as a valid player.
        (i.e., `player` should be either game.__player_1__ or
        game.__player_2__).
    Returns
    ----------
    float
        The heuristic value of the current game state
    """
   
    # Same as open_move_score
    if game.is_loser(player):
        return float("-inf")

    if game.is_winner(player):
        return float("inf")
    
    # if no one has one yet, get the available moves for each and return the difference
    own_moves = len(game.get_legal_moves(player))
    opp_moves = len(game.get_legal_moves(game.get_opponent(player)))
    return float(own_moves - opp_moves)

### Players

In [37]:
class RandomPlayer():
    """Player that chooses a move randomly."""

    def get_move(self, game, legal_moves, time_left):
        """Randomly select a move from the available legal moves.
        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).
        legal_moves : list<(int, int)>
            A list containing legal moves. Moves are encoded as tuples of pairs
            of ints defining the next (row, col) for the agent to occupy.
        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.
            
        Returns
        ----------
        (int, int)
            A randomly selected legal move; may return (-1, -1) if there are
            no available legal moves.
        """

        # Pick randomly from legal_moves, else return (-1,-1)
        if not legal_moves:
            return (-1, -1)
        return legal_moves[randint(0, len(legal_moves) - 1)]


class GreedyPlayer():
    """Player that chooses next move to maximize heuristic score. This is
    equivalent to a minimax search agent with a search depth of one.
    """
    
    # initialize the class by picking a certain scoring fn from above
    def __init__(self, score_fn=open_move_score):
        self.score = score_fn
        
    # same method as RandomPlayer() but with max instead of random
    def get_move(self, game, legal_moves, time_left):
        """Select the move from the available legal moves with the highest
        heuristic score.
        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).
        legal_moves : list<(int, int)>
            A list containing legal moves. Moves are encoded as tuples of pairs
            of ints defining the next (row, col) for the agent to occupy.
        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.
        Returns
        ----------
        (int, int)
            The move in the legal moves list with the highest heuristic score
            for the current game state; may return (-1, -1) if there are no
            legal moves.
        """
        
        # no legal moves, loses
        if not legal_moves:
            return (-1, -1)
        # score all the potential moves use self.score and forecast_move for every legal move
        _, move = max([(self.score(game.forecast_move(m), self), m) for m in legal_moves])
        # ignore the score, return the best move 
        return move


class HumanPlayer():
    """Player that chooses a move according to user's input."""

    def get_move(self, game, legal_moves, time_left):
        """
        Select a move from the available legal moves based on user input at the
        terminal.
        
        **********************************************************************
        NOTE: If testing with this player, remember to disable move timeout in
              the call to `Board.play()`.
        **********************************************************************
       
       Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).
        legal_moves : list<(int, int)>
            A list containing legal moves. Moves are encoded as tuples of pairs
            of ints defining the next (row, col) for the agent to occupy.
        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.
        Returns
        ----------
        (int, int)
            The move in the legal moves list selected by the user through the
            terminal prompt; automatically return (-1, -1) if there are no
            legal moves
        """
        
        if not legal_moves:
            return (-1, -1)

        print(('\t'.join(['[%d] %s' % (i, str(move)) for i, move in enumerate(legal_moves)])))

        valid_choice = False
        while not valid_choice:
            try:
                index = int(input('Select move index:'))
                valid_choice = 0 <= index < len(legal_moves)

                if not valid_choice:
                    print('Illegal move! Try again.')

            except ValueError:
                print('Invalid index! Try again.')
        
        print('Giving you back {}'.format(legal_moves[index]))
        return legal_moves[index]

## Practice Game

In [166]:
# from isolation.py get the Board class
from isolation import Board
from random import randint 

## Setup
# create an isolation board (by default 7x7) and some players
player1 = GreedyPlayer()
player2 = CustomPlayer()
game = Board(player1, player2)

# place player 1 on the board at row 2, column 3, then place player 2 on
# the board at row 0, column 5; display the resulting board state.  Note
# that .apply_move() changes the calling object

# apply move and print the game board
game.apply_move((2, 3))
game.apply_move((0, 5))
print(game.to_string())

## Play

# players take turns moving on the board, so player1 should be next to move
assert(player1 == game.active_player)

# get a list of the legal moves available to the active player
print("Legal moves:\n{!s} for player {!s}".format(game.get_legal_moves(),game.active_player))

# get a successor of the current state by making a copy of the board and
# applying a move. Notice that this does NOT change the calling object
# (unlike .apply_move()).

# make a new board as a branch of the old board, and assert not the same as before, then print both 
new_game = game.forecast_move((1, 1))
assert(new_game.to_string() != game.to_string())
print("\nOld state:\n{}".format(game.to_string()))
print("\nNew state:\n{}".format(new_game.to_string()))

# play the remainder of the game automatically -- outcome can be "illegal
# move" or "timeout"; it should _always_ be "illegal move" in this example

# play the game, and get the winner, history, and outcome from 
winner, history, outcome = game.play(time_limit=100)
print("\nWinner: {}\nOutcome: {}".format(winner, outcome))
print(game.to_string())
print("Move history:\n{!s}".format(history))

 |   |   |   |   |   | 2 |   | 
 |   |   |   |   |   |   |   | 
 |   |   |   | 1 |   |   |   | 
 |   |   |   |   |   |   |   | 
 |   |   |   |   |   |   |   | 
 |   |   |   |   |   |   |   | 
 |   |   |   |   |   |   |   | 

Legal moves:
[(0, 2), (0, 4), (1, 1), (1, 5), (3, 1), (3, 5), (4, 2), (4, 4)] for player <__main__.GreedyPlayer object at 0x1049bac88>

Old state:
 |   |   |   |   |   | 2 |   | 
 |   |   |   |   |   |   |   | 
 |   |   |   | 1 |   |   |   | 
 |   |   |   |   |   |   |   | 
 |   |   |   |   |   |   |   | 
 |   |   |   |   |   |   |   | 
 |   |   |   |   |   |   |   | 


New state:
 |   |   |   |   |   | 2 |   | 
 |   | 1 |   |   |   |   |   | 
 |   |   |   | - |   |   |   | 
 |   |   |   |   |   |   |   | 
 |   |   |   |   |   |   |   | 
 |   |   |   |   |   |   |   | 
 |   |   |   |   |   |   |   | 


Winner: <__main__.CustomPlayer object at 0x1049baef0>
Outcome: illegal move
 | - |   |   |   | - | - |   | 
 |   |   | - | - | - |   | 2 | 
 |   | - | - | - | - |   


The steps below outline one suggested process for completing the project -- however, this is just a suggestion to help you get started.  Unit tests can be executed by running `python agent_test.py -v`.  (See the [unittest](https://docs.python.org/3/library/unittest.html#basic-example) module for details.)

0. Pass the test_get_move_interface and test_minimax_interface unit tests by implementing a fixed-depth call to minimax in `CustomPlayer.get_move()` and implementing a single-level search in `CustomPlayer.minimax()` (the interface checks only tests depth=1)

0. Pass the test_minimax test by extending your `CustomPlayer.minimax()` function with the full recursive search process.  See Also: [AIMA Minimax Decision](https://github.com/aimacode/aima-pseudocode/blob/master/md/Minimax-Decision.md)

0. Pass the test_alphabeta_interface test by copying the code from `CustomPlayer.minimax()` into the `CustomPlayer.alphabeta()` function.

0. Pass the test_alphabeta test by extending your `CustomPlayer.alphabeta()` function to include alpha and beta pruning.  See Also: [AIMA Alpha-Beta Search](https://github.com/aimacode/aima-pseudocode/blob/master/md/Alpha-Beta-Search.md)

0. Pass the test_get_move test by extending your fixed-depth call in `CustomPlayer.get_move()` to implement Iterative Deepening.  See Also [AIMA Iterative Deepening Search](https://github.com/aimacode/aima-pseudocode/blob/master/md/Iterative-Deepening-Search.md)

0. Finally, pass the test_heuristic test by implementing any heuristic in `custom_score()`.  (This test only validates the return value type -- it does not check for "correctness" of your heuristic.)  You can see example heuristics in the `sample_players.py` file.

In [108]:
"""This file contains all the classes you must complete for this project.

You can use the test cases in agent_test.py to help during development, and
augment the test suite with your own test cases to further test your code.

You must test your agent's strength against a set of agents with known
relative strength using tournament.py and include the results in your report.
"""
import random
import isolation # local isolation.py for board variables 

# A given class that yields the Timeout exception

class Timeout(Exception):
    """Subclass base exception for code clarity."""
    pass

## Custom Score

In [163]:
def custom_score(game, player):
    """Calculate the heuristic value of a game state from the point of view
    of the given player.

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    player : object
        A player instance in the current game (i.e., an object corresponding to
        one of the player objects `game.__player_1__` or `game.__player_2__`.)

    Returns
    ----------
    float
        The heuristic value of the current game state to the specified player.
    """

    
    # Heuristic 1: Available moves
    # Same as one in the material, i.e. #my_moves 
    def open_moves(myMoves):
        if game.is_loser(player):
            return float("-inf")

        if game.is_winner(player):
            return float("+inf")

        return float(len(myMoves))
    
    # Heuristic 2: Available moves less opponent moves 
    # Borrowed from sample_players.py, yields the #my_moves - #opponent_moves eval fn
    def difference_moves(myMoves, opponentMoves):
        # All legal moves are equal. We calculate the difference in the number
        # of legal moves between the player and the opponent
        if game.is_loser(player):
            return float("-inf")

        if game.is_winner(player):
            return float("+inf")
        
        #print('You have {} moves, they have {} moves'.format(myMoves, opponentMoves))
        return float(len(myMoves) - len(opponentMoves))


    # Heuristic 3: Quadratic version of Heuristic 2
    # Similar to above, but squares both side to account for negatives. 
    def quadratic_difference_moves(myMoves, opponentMoves):
        if game.is_loser(player):
            return float("-inf")

        if game.is_winner(player):
            return float("+inf")

        return float(len(myMoves) * len(myMoves) -
                     len(opponentMoves) * len(opponentMoves))


    # Heuristic 4: Final weighted average of all 3 heuristics 
    # Assign weights to all the heuristics, and calculate the weighted average.
    def final_score(heur1_weight,
                    heur2_weight, 
                    heur3_weight):
        
        if game.is_loser(player):
            return float("-inf")

        if game.is_winner(player):
            return float("+inf")
        
        # check the weights! 
        if heur1_weight + heur2_weight + heur3_weight != 1.0:
            raise ValueError("the weights in the do not add up to 1; check your weights!")
        else: 
            # everything's fine
            weights = [heur1_weight, heur2_weight, heur3_weight] 
        
        # which heuristics are you using? 
        functions = [difference_moves(myMoves, opponentMoves),
                     quadratic_difference_moves(myMoves, opponentMoves),
                     open_moves(myMoves)]
        
        # get the weighted score
        return (sum([i*j for i, j in zip(weights, functions)]))

    

    # find opponent
    opponent = game.get_opponent(player)
    
    # determine the opponent's legal moves and the player's legal moves
    opponentMoves = game.get_legal_moves(opponent)
    myMoves = game.get_legal_moves(player)
    
    # return the final score (weights must add up to 1) 
    return final_score(0.4, 0.3, 0.3)

In [ ]:
# Other heuristics? 

    # Hueristic 1
    # All legal moves are not equal. I assign a weight of 1/2 to moves around
    # a corridor which is one square thick around the board, i.e.
    # for 1<row<n-2 and 1<col<n-2, the move is assigned a weight of
    # (1- discount). For the rest of the board, the move is assigned
    # a weight of 1.0
    def weighted_nb_legal_moves(moves, discount):
        if game.is_loser(player):
            return float("-inf")

        if game.is_winner(player):
            return float("+inf")

        weight = []
        for move in moves:
            if (move[0] == 0) or (move[0] == game.width - 1) or \
               (move[0] in(0, 1) and move[1] > 1 and
                    move[1] < game.height - 2) or\
               (move[0] == game.width - 2 and move[1] > 2 and
                    move[1] < game.height - 2):
                    weight.append(1 - discount)
            else:
                weight.append(1.0)
        res = sum(weight)
        return res

    def weighted_difference(discount, myMoves, opponentMoves):
        return weighted_nb_legal_moves(myMoves, discount) - \
                weighted_nb_legal_moves(opponentMoves, discount)

    # Final Heuristic
    # Here, we take the minimum of heuristic 1, 2, open_moves and
    # improved_score.
    def final_custom_score_min(discount):
        if game.is_loser(player):
            return float("-inf")

        if game.is_winner(player):
            return float("+inf")

        if discount > 1 or discount < 0:
            print("The discount in the custom_score function must be " +
                  "between 0 and 1, included.")
        functions = [difference_moves(myMoves, opponentMoves),
                     weighted_difference(discount, myMoves, opponentMoves),
                     quadratic_difference(myMoves, opponentMoves),
                     open_moves(myMoves)]
        return min(functions)

### Custom Player

#### The Example

In [167]:
type(1)

int

In [165]:
class CustomPlayer:
    """Game-playing agent that chooses a move using a given evaluation function
    and a depth-limited minimax algorithm with alpha-beta pruning. The goal is to finish
    and test this player to make sure it returns a good move before the search time limit expires.

    Parameters
    ----------
    search_depth : int (optional)
        A strictly positive integer (i.e., 1, 2, 3,...) for the number of
        plys in the game tree to explore for search. 

    score_fn : callable (optional)
        A function to use for heuristic evaluation of game states.

    iterative : boolean (optional)
        Flag indicating whether to perform fixed-depth search (False) or
        iterative deepening search (True).

    method : {'minimax', 'alphabeta'} (optional)
        The name of the search method to use in get_move().

    timeout : float (optional)
        Time remaining (in milliseconds) when search is aborted. Should be a
        positive value large enough to allow the function to return before the
        timer expires.
    """

    # initialize the agent with a depth 3, the custom_score eval fn, iterative search
    # via the minimax method, and a 10 second timeout
    def __init__(self, search_depth=3, score_fn=custom_score,
                 iterative=True, method='minimax', timeout=10.):
        self.search_depth = search_depth
        self.iterative = iterative
        self.score = score_fn
        self.method = method
        self.time_left = None
        self.TIMER_THRESHOLD = timeout

    def get_move(self, game, legal_moves, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.

        This function must perform iterative deepening if self.iterative=True,
        and it must use the search method (minimax or alphabeta) corresponding
        to the self.method value.

        **********************************************************************
        NOTE: If time_left < 0 when this function returns, the agent will
              forfeit the game due to timeout. You must return _before_ the
              timer reaches 0.
        **********************************************************************

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        legal_moves : list<(int, int)>
            A list containing legal moves. Moves are encoded as tuples of pairs
            of ints defining the next (row, col) for the agent to occupy.

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        ----------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """

        self.time_left = time_left
        
        # Perform any required initializations (i.e., an opening book)
        
        # Opening book
        # TODO: open_book = []? # best starting moves? 
    
        # Setup
        # Lets make a list to recieve all potential moves suggested from the methods
        potential_moves = list()
        
        # Move Check 
        # Returning immediately if there are no legal moves
        if len(legal_moves) == 0:
            return (-1, -1)

        try:
            # The search method call (alpha beta or minimax) should happen in
            # here in order to avoid timeout. The try/except block will
            # automatically catch the exception raised by the search method
            # when the timer gets close to expiring
            
            # Search Depth            
            # If depth is not 0, make it 1
            if self.search_depth < 0:
                self.search_depth = 1

            # If iterative, pick a method and go to the given depth using that method
            if self.iterative:
                if self.method == 'minimax':
                    # if minimax, append potential_moves with our best moves
                    for d in range(1, self.search_depth + 1):
                        potential_moves.append(self.minimax(game, d))
                elif self.method == 'alphabeta':
                    # if alphabeta, same idea 
                    for d in range(1, self.search_depth + 1):
                        potential_moves.append(self.alphabeta(game, d))
            
            # if it's not iterative, just grab the closest move and go
            else:
                # minimax for the current ply
                if self.method == 'minimax':
                    potentialMoves.append(self.minimax(game,
                                                       self.search_depth))
                
                # alphabeta for the current ply 
                elif self.method == 'alphabeta':
                    potentialMoves.append(self.alphabeta(game,
                                                         self.search_depth))
        
        # if we're out of time, raise the Timeout Exception 
        except Timeout:
            pass

        # if we still have time, and potential_moves, grab the best move from the last completed search
        if potential_moves != []:
            best_move = max(potential_moves)
            return best_move[1]
        else:
            # lol fail
            return (-1, -1)

    def minimax(self, game, depth, maximizing_player=True):
        """Implement the minimax search algorithm as described in the lectures.
        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state
        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting
        maximizing_player : bool
            Flag indicating whether the current search depth corresponds to a
            maximizing layer (True) or a minimizing layer (False)
        Returns
        -------
        float
            The score for the current search branch
        tuple(int, int)
            The best move for the current branch; (-1, -1) for no legal moves
        Notes
        -----
            (1) You MUST use the `self.score()` method for board evaluation
                to pass the project unit tests; you cannot call any other
                evaluation function directly.
        """
        
        # Check if we're out of time
        if self.time_left() < self.TIMER_THRESHOLD:
            raise Timeout()

        # Setup
        scores = []
        final_move = (-1, -1)

        # Which player?
        if maximizing_player:
            # if it's the active player, he wants the maximum score (whichever player)
            player = game.active_player
            final_score = float("-inf")
        else:
            # if it's the inactive player, he wants the minimum score (whichever player)
            player = game.inactive_player
            final_score = float("+inf")

        # Let's get all of his moves 
        legal_moves = game.get_legal_moves()

        # if no moves, deafault to that (-1,-1)
        if len(legal_moves) == 0:
            score, selectedMove = self.score(game, player), (-1, -1)

        # if only 1 move deep, forecast just the next game for all legal moves 
        if depth == 1:
            for move in legal_moves:
                nextGame = game.forecast_move(move) # forecast the move! neat function.
                score = self.score(nextGame, player) # check the score of that game, using that players score fn
                scores.append(score) # append that score to the list we made earlier
                
                # if that score is better that +/- inf (for max or min player) use that!
                if (maximizing_player and score > final_score) or (not maximizing_player and score < final_score):
                            final_score, final_move = score, move     
                        
        # if we can go deeper
        elif depth > 1:
            
            # for every legal move
            for move in legal_moves:
                # forecast the next game for a given
                nextGame = game.forecast_move(move)
                
                # call minimax on itself, get the score of the next game to the final score
                # because of depth - 1, will iterate until we hit depth = 1
                # then return the last thing in that new list (score of the latest game)
                score = self.minimax(nextGame, depth - 1, not maximizing_player)[0]
                scores.append(score)
        
                # if that score is better that +/- inf (for max or min player) use that!
                if (maximizing_player and score > final_score) or (not maximizing_player and score < final_score):
                            final_score, final_move = score, move
            
        # return the result
        #print('\nCurrent player is maximizing? {}'.format(maximizing_player))
        #print('Score is {} from {}, at depth {}'.format(score, scores, depth))
        return final_score, final_move
    
    
    def alphabeta(self, game, depth, alpha=float("-inf"), beta=float("inf"),
                  maximizing_player=True):
        """Implement minimax search with alpha-beta pruning as described in the
        lectures.

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        alpha : float
            Alpha limits the lower bound of search on minimizing layers

        beta : float
            Beta limits the upper bound of search on maximizing layers

        maximizing_player : bool
            Flag indicating whether the current search depth corresponds to a
            maximizing layer (True) or a minimizing layer (False)

        Returns
        ----------
        float
            The score for the current search branch

        tuple(int, int)
            The best move for the current branch; (-1, -1) for no legal moves
        """
        
        # Check if we're out of time
        if self.time_left() < self.TIMER_THRESHOLD:
            raise Timeout()
        
        # Setup
        scores = []
        explore = True
        final_move = (-1, -1)
        
        # Use the minimax logic from before to set ranges for maxing and minimizing respectively 
        if maximizing_player:
            player = game.active_player
            final_score = float("-inf")
        else:
            player = game.inactive_player
            final_score = float("+inf")
            
        # return score and (-1, -1) if we're out of moves so we can lose
        legal_moves = game.get_legal_moves()
        if len(legal_moves) == 0:
            return (self.score(game, player), (-1, -1))
        
        # for depth = 1
        if depth == 1:
            for move in legal_moves:
                if explore:
                    # check the next game in the tree
                    nextGame = game.forecast_move(move)
                    # find that games score
                    score = self.score(nextGame, player)
                    # add that to the score list
                    scores.append(score)
                    
                    # if we're minimizing update the beta, and get us that score and move
                    if not maximizing_player:
                        beta = min(beta, score) # get the smallest score; that's the beta, prune the rest
                        if score < final_score:
                            final_move = move
                            final_score = score
                    # if we're maximizing update alpha, and get us that score and move
                    else:
                        alpha = max(alpha, score) # get the largest score; that's the alpha, prune the rest
                        if score > final_score:
                            final_move = move
                            final_score = score
                            
                    # if alpha is greater than beta, stop searching; we're lost
                    if alpha >= beta:
                        explore = False
                        
            return final_score, final_move
        
        # now for all the depths!
        elif depth > 1:
            for move in legal_moves:
                if explore:
                    nextGame = game.forecast_move(move)
                    # let's use that recursive trick to call ourselves on the next depth over and over until depth = 1
                    score = self.alphabeta(nextGame, depth - 1,
                                           alpha,
                                           beta, not maximizing_player)[0]
                    scores.append(score)
                    
                    # do exactly what we did above, but now with recursion! (Since this calls for any depth > 1)
                    if not maximizing_player:
                        beta = min(beta, score)
                        if score < final_score:
                            final_move = move
                            final_score = score
                            
                    else:
                        alpha = max(alpha, score)
                        if score > final_score:
                            final_move = move
                            final_score = score
                            
                    # interrupt again if alpha >= beta
                    if alpha >= beta:
                        explore = False
            
            # return results!
            return final_score, final_move

#### Minimax

# MINIMAX-DECISION

## AIMA3e
__function__ MINIMAX-DECISION(_state_) __returns__ _an action_  
&emsp;__return__ arg max<sub> _a_ &Element; ACTIONS(_s_)</sub> MIN\-VALUE(RESULT(_state_, _a_))  

---
__function__ MAX\-VALUE(_state_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __the return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_)))  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __the return__ UTILITY(_state_)  
&emsp;_v_ &larr; &infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_)))  
&emsp;__return__ _v_  

---
__Figure__ ?? An algorithm for calculating minimax decisions. It returns the action corresponding to the best possible move, that is, the move that leads to the outcome with the best utility, under the assumption that the opponent plays to minimize utility. The functions MAX\-VALUE and MIN\-VALUE go through the whole game tree, all the way to the leaves, to determine the backed\-up value of a state. The notation argmax <sub>_a_ &Element; _S_</sub> _f_(_a_) computes the element _a_ of set _S_ that has maximum value of _f_(_a_).

In [136]:
def minimax(self, game, depth, maximizing_player=True):
    """Implement the minimax search algorithm as described in the lectures.
    Parameters
    ----------
    game : isolation.Board
        An instance of the Isolation game `Board` class representing the
        current game state
    depth : int
        Depth is an integer representing the maximum number of plies to
        search in the game tree before aborting
    maximizing_player : bool
        Flag indicating whether the current search depth corresponds to a
        maximizing layer (True) or a minimizing layer (False)
    Returns
    -------
    float
        The score for the current search branch
    tuple(int, int)
        The best move for the current branch; (-1, -1) for no legal moves
    Notes
    -----
        (1) You MUST use the `self.score()` method for board evaluation
            to pass the project unit tests; you cannot call any other
            evaluation function directly.
    """
    if self.time_left() < self.TIMER_THRESHOLD:
        raise Timeout()

    # setup
    scores = []
    selected_move = (-1, -1)
    
    # which player?
    if maximizing_player:
        # max?
        player = game.active_player
        final_score = float("-inf")
    else:
        # min?
        player = game.inactive_player
        final_score = float("+inf")

    # what moves? 
    legal_moves = game.get_legal_moves()

    # if no moves
    if len(legal_moves) == 0:
        # (-1, -1)
        score, selectedMove = self.score(game, player), (-1, -1)

    # if only 1 move deep
    if depth == 1:
        for move in legal_moves:
            nextGame = game.forecast_move(move)
            score = self.score(nextGame, player)
            scores.append(score)
            if (maximizing_player and score > scoreFinal) or \
               (not maximizing_player and score < scoreFinal):
                scoreFinal, selectedMove = score, move
        return scoreFinal, selectedMove

    elif depth > 1:
        for move in legal_moves:
            nextGame = game.forecast_move(move)
            score = self.minimax(nextGame, depth - 1,
                                 not maximizing_player)[0]
            scores.append(score)
            if (maximizing_player and score > scoreFinal) or \
               (not maximizing_player and score < scoreFinal):
                scoreFinal, selectedMove = score, move
    return scoreFinal, selectedMove

In [ ]:
def minimax(self, game, depth, maximizing_player=True):
    """Implement the minimax search algorithm as described in the lectures.

    Parameters
    ----------
    game : isolation.Board
        An instance of the Isolation game `Board` class representing the
        current game state

    depth : int
        Depth is an integer representing the maximum number of plies to
        search in the game tree before aborting

    maximizing_player : bool
        Flag indicating whether the current search depth corresponds to a
        maximizing layer (True) or a minimizing layer (False)

    Returns
    ----------
    float
        The score for the current search branch

    tuple(int, int)
        The best move for the current branch; (-1, -1) for no legal moves
    """
    if self.time_left() < self.TIMER_THRESHOLD:
        raise Timeout()
    # initialize
    scores = []
    selectedMove = (-1, -1)
    # determine the player
    if maximizing_player:
        player = game.active_player
        scoreFinal = float("-inf")
    else:
        player = game.inactive_player
        scoreFinal = float("+inf")

    legal_moves = game.get_legal_moves()

    if len(legal_moves) == 0:
        score, selectedMove = self.score(game, player), (-1, -1)

    if depth == 1:
        for move in legal_moves:
            nextGame = game.forecast_move(move)
            score = self.score(nextGame, player)
            scores.append(score)
            if (maximizing_player and score > scoreFinal) or \
               (not maximizing_player and score < scoreFinal):
                scoreFinal, selectedMove = score, move
        return scoreFinal, selectedMove

    elif depth > 1:
        for move in legal_moves:
            nextGame = game.forecast_move(move)
            score = self.minimax(nextGame, depth - 1,
                                 not maximizing_player)[0]
            scores.append(score)
            if (maximizing_player and score > scoreFinal) or \
               (not maximizing_player and score < scoreFinal):
                scoreFinal, selectedMove = score, move
    return scoreFinal, selectedMove

#### Alpha Beta

# ALPHA-BETA-SEARCH

## AIMA3e
__function__ ALPHA-BETA-SEARCH(_state_) __returns__ an action  
&emsp;_v_ &larr; MAX\-VALUE(_state_, &minus;&infin;, &plus;&infin;)  
&emsp;__return__ the _action_ in ACTIONS(_state_) with value _v_  

---
__function__ MAX\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __the return__ UTILITY(_state_)  
&emsp;_v_ &larr; &minus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MAX(_v_, MIN\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &ge; _&beta;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&alpha;_ &larr; MAX(_&alpha;_, _v_)  
&emsp;__return__ _v_  

---
__function__ MIN\-VALUE(_state_, _&alpha;_, _&beta;_) __returns__ _a utility value_  
&emsp;__if__ TERMINAL\-TEST(_state_) __the return__ UTILITY(_state_)  
&emsp;_v_ &larr; &plus;&infin;  
&emsp;__for each__ _a_ __in__ ACTIONS(_state_) __do__  
&emsp;&emsp;&emsp;_v_ &larr; MIN(_v_, MAX\-VALUE(RESULT(_state_, _a_), _&alpha;_, _&beta;_))  
&emsp;&emsp;&emsp;__if__ _v_ &le; _&alpha;_ __then return__ _v_  
&emsp;&emsp;&emsp;_&beta;_ &larr; MIN(_&beta;_, _v_)  
&emsp;__return__ _v_  


---
__Figure__ ?? The alpha\-beta search algorithm. Notice that these routines are the same as the MINIMAX functions in Figure ??, except for the two lines in each of MIN\-VALUE and MAX\-VALUE that maintain _&alpha;_ and _&beta;_ (and the bookkeeping to pass these parameters along).

In [ ]:
    def alphabeta(self, game, depth, alpha=float("-inf"), beta=float("inf"),
                  maximizing_player=True):
        """Implement minimax search with alpha-beta pruning as described in the
        lectures.

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        alpha : float
            Alpha limits the lower bound of search on minimizing layers

        beta : float
            Beta limits the upper bound of search on maximizing layers

        maximizing_player : bool
            Flag indicating whether the current search depth corresponds to a
            maximizing layer (True) or a minimizing layer (False)

        Returns
        ----------
        float
            The score for the current search branch

        tuple(int, int)
            The best move for the current branch; (-1, -1) for no legal moves
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise Timeout()
        # initialize
        scores = []
        explore = True
        selectedMove = (-1, -1)
        # select player and initialize the resulting score
        if maximizing_player:
            player = game.active_player
            scoreFinal = float("-inf")
        else:
            player = game.inactive_player
            scoreFinal = float("+inf")
        # return the score and (-1, -1) if there are no legal moves
        legal_moves = game.get_legal_moves()
        if len(legal_moves) == 0:
            return (self.score(game, player), (-1, -1))
        # calculate the score for depth = 1
        if depth == 1:
            for move in legal_moves:
                if explore:
                    nextGame = game.forecast_move(move)
                    score = self.score(nextGame, player)
                    scores.append(score)
                    # update beta, the final score and the selected move
                    if not maximizing_player:
                        beta = min(beta, score)
                        if score < scoreFinal:
                            selectedMove = move
                            scoreFinal = score
                    # update alpha, the final score and the selected move
                    else:
                        alpha = max(alpha, score)
                        if score > scoreFinal:
                            selectedMove = move
                            scoreFinal = score
                    # interrupt the search if alpha >= beta
                    if alpha >= beta:
                        explore = False
            return scoreFinal, selectedMove
        # proceed with recursion with depth > 1
        elif depth > 1:
            for move in legal_moves:
                if explore:
                    nextGame = game.forecast_move(move)
                    score = self.alphabeta(nextGame, depth - 1,
                                           alpha,
                                           beta, not maximizing_player)[0]
                    scores.append(score)
                    # update beta, the final score and the selected move
                    if not maximizing_player:
                        beta = min(beta, score)
                        if score < scoreFinal:
                            selectedMove = move
                            scoreFinal = score
                    # update alpha, the final score and the selected move
                    else:
                        alpha = max(alpha, score)
                        if score > scoreFinal:
                            selectedMove = move
                            scoreFinal = score
                    # interrupt the search if alpha >= beta
                    if alpha >= beta:
                        explore = False
            # return result
            return scoreFinal, selectedMove

#### Iterative Deepening

# ITERATIVE-DEEPENING-SEARCH

## AIMA3e
__function__ ITERATIVE-DEEPENING-SEARCH(_problem_) __returns__ a solution, or failure  
&emsp;__for__ _depth_ = 0 to &infin; __do__  
&emsp;&emsp;&emsp;_result_ &larr; DEPTH\-LIMITED\-SEARCH(_problem_,_depth_)  
&emsp;&emsp;&emsp;__if__ _result_ &ne; cutoff __then return__ _result_

---
__Figure__ ?? The iterative deepening search algorithm, which repeatedly applies depth\-limited search with increasing limits. It terminates when a solution is found or if the depth\-limited search returns _failure_, meaning that no solution exists.

In [ ]:
def get_move(self, game, legal_moves, time_left):
    """Search for the best move from the available legal moves and return a
    result before the time limit expires.

    This function must perform iterative deepening if self.iterative=True,
    and it must use the search method (minimax or alphabeta) corresponding
    to the self.method value.

    **********************************************************************
    NOTE: If time_left < 0 when this function returns, the agent will
          forfeit the game due to timeout. You must return _before_ the
          timer reaches 0.
    **********************************************************************

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    legal_moves : list<(int, int)>
        A list containing legal moves. Moves are encoded as tuples of pairs
        of ints defining the next (row, col) for the agent to occupy.

    time_left : callable
        A function that returns the number of milliseconds left in the
        current turn. Returning with any less than 0 ms remaining forfeits
        the game.

    Returns
    ----------
    (int, int)
        Board coordinates corresponding to a legal move; may return
        (-1, -1) if there are no available legal moves.
    """

    self.time_left = time_left
    # Perform any required initializations, including selecting an initial
    # move from the game board (i.e., an opening book), or returning
    # immediately if there are no legal moves
    potentialMoves = list()
    if len(legal_moves) == 0:
        return (-1, -1)

    try:
        # The search method call (alpha beta or minimax) should happen in
        # here in order to avoid timeout. The try/except block will
        # automatically catch the exception raised by the search method
        # when the timer gets close to expiring
        if self.search_depth < 0:
            self.search_depth = int(1e10)

        if self.iterative:
            if self.method == 'minimax':
                for d in range(1, self.search_depth + 1):
                    potentialMoves.append(self.minimax(game, d))
            elif self.method == 'alphabeta':
                for d in range(1, self.search_depth + 1):
                    potentialMoves.append(self.alphabeta(game, d))
        else:
            if self.method == 'minimax':
                potentialMoves.append(self.minimax(game,
                                                   self.search_depth))
            elif self.method == 'alphabeta':
                potentialMoves.append(self.alphabeta(game,
                                                     self.search_depth))

    except Timeout:
        pass

    # Return the best move from the last completed search iteration
    if potentialMoves != []:
        bestMove = max(potentialMoves)
        return bestMove[1]
    else:
        return (-1, -1)

In [ ]:
def get_move(self, game, legal_moves, time_left):
    """Search for the best move from the available legal moves and return a
    result before the time limit expires.

    This function must perform iterative deepening if self.iterative=True,
    and it must use the search method (minimax or alphabeta) corresponding
    to the self.method value.

    **********************************************************************
    NOTE: If time_left < 0 when this function returns, the agent will
          forfeit the game due to timeout. You must return _before_ the
          timer reaches 0.
    **********************************************************************

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    legal_moves : list<(int, int)>
        A list containing legal moves. Moves are encoded as tuples of pairs
        of ints defining the next (row, col) for the agent to occupy.

    time_left : callable
        A function that returns the number of milliseconds left in the
        current turn. Returning with any less than 0 ms remaining forfeits
        the game.

    Returns
    ----------
    (int, int)
        Board coordinates corresponding to a legal move; may return
        (-1, -1) if there are no available legal moves.
    """

    self.time_left = time_left

    # Perform any required initializations (i.e., an opening book)

    # Opening book
    # TODO: open_book = []? # best starting moves? 

    # Setup
    # Lets make a list to recieve all potential moves suggested from the methods
    potential_moves = list()

    # Move Check 
    # Returning immediately if there are no legal moves
    if len(legal_moves) == 0:
        return (-1, -1)

    try:
        # The search method call (alpha beta or minimax) should happen in
        # here in order to avoid timeout. The try/except block will
        # automatically catch the exception raised by the search method
        # when the timer gets close to expiring

        # Search Depth            
        # If depth is not 0, make it 1
        if self.search_depth < 0:
            self.search_depth = 1

        # If iterative, pick a method and go to the given depth using that method
        if self.iterative:
            if self.method == 'minimax':
                # if minimax, append potential_moves with our best moves
                for d in range(1, self.search_depth + 1):
                    potential_moves.append(self.minimax(game, d))
            elif self.method == 'alphabeta':
                # if alphabeta, same idea 
                for d in range(1, self.search_depth + 1):
                    potential_moves.append(self.alphabeta(game, d))

        # if it's not iterative, just grab the closest move and go
        else:
            # minimax for the current ply
            if self.method == 'minimax':
                potential_moves.append(self.minimax(game,
                                                   self.search_depth))

            # alphabeta for the current ply 
            elif self.method == 'alphabeta':
                potential_moves.append(self.alphabeta(game,
                                                     self.search_depth))

    # if we're out of time, raise the Timeout Exception 
    except Timeout:
        pass

    # if we still have time, and potential_moves, grab the best move from the last completed search
    if potential_moves != []:
        best_move = max(potential_moves)
        return best_move[1]
    else:
        # lol fail
        return (-1, -1)

In [ ]:
def get_move(self, game, legal_moves, time_left):
    """Search for the best move from the available legal moves and return a
    result before the time limit expires.

    This function must perform iterative deepening if self.iterative=True,
    and it must use the search method (minimax or alphabeta) corresponding
    to the self.method value.

    **********************************************************************
    NOTE: If time_left < 0 when this function returns, the agent will
          forfeit the game due to timeout. You must return _before_ the
          timer reaches 0.
    **********************************************************************

    Parameters
    ----------
    game : `isolation.Board`
        An instance of `isolation.Board` encoding the current state of the
        game (e.g., player locations and blocked cells).

    legal_moves : list<(int, int)>
        A list containing legal moves. Moves are encoded as tuples of pairs
        of ints defining the next (row, col) for the agent to occupy.

    time_left : callable
        A function that returns the number of milliseconds left in the
        current turn. Returning with any less than 0 ms remaining forfeits
        the game.

    Returns
    ----------
    (int, int)
        Board coordinates corresponding to a legal move; may return
        (-1, -1) if there are no available legal moves.
    """

    self.time_left = time_left
    # Perform any required initializations, including selecting an initial
    # move from the game board (i.e., an opening book), or returning
    # immediately if there are no legal moves
    potentialMoves = list()
    if len(legal_moves) == 0:
        return (-1, -1)

    try:
        # The search method call (alpha beta or minimax) should happen in
        # here in order to avoid timeout. The try/except block will
        # automatically catch the exception raised by the search method
        # when the timer gets close to expiring
        if self.search_depth < 0:
            self.search_depth = int(1e10)

        if self.iterative:
            if self.method == 'minimax':
                for d in range(1, self.search_depth + 1):
                    potentialMoves.append(self.minimax(game, d))
            elif self.method == 'alphabeta':
                for d in range(1, self.search_depth + 1):
                    potentialMoves.append(self.alphabeta(game, d))
        else:
            if self.method == 'minimax':
                potentialMoves.append(self.minimax(game,
                                                   self.search_depth))
            elif self.method == 'alphabeta':
                potentialMoves.append(self.alphabeta(game,
                                                     self.search_depth))

    except Timeout:
        pass

    # Return the best move from the last completed search iteration
    if potentialMoves != []:
        bestMove = max(potentialMoves)
        return bestMove[1]
    else:
        return (-1, -1)

# Given

In [ ]:
class CustomPlayer:
    """Game-playing agent that chooses a move using a given evaluation function
    and a depth-limited minimax algorithm with alpha-beta pruning. The goal is to finish
    and test this player to make sure it returns a good move before the search time limit expires.

    Parameters
    ----------
    search_depth : int (optional)
        A strictly positive integer (i.e., 1, 2, 3,...) for the number of
        plys in the game tree to explore for search. 

    score_fn : callable (optional)
        A function to use for heuristic evaluation of game states.

    iterative : boolean (optional)
        Flag indicating whether to perform fixed-depth search (False) or
        iterative deepening search (True).

    method : {'minimax', 'alphabeta'} (optional)
        The name of the search method to use in get_move().

    timeout : float (optional)
        Time remaining (in milliseconds) when search is aborted. Should be a
        positive value large enough to allow the function to return before the
        timer expires.
    """

    # initialize the agent with a depth 3, the custom_score eval fn, iterative search
    # via the minimax method, and a 10 second timeout
    def __init__(self, search_depth=3, score_fn=custom_score,
                 iterative=True, method='minimax', timeout=10.):
        self.search_depth = search_depth
        self.iterative = iterative
        self.score = score_fn
        self.method = method
        self.time_left = None
        self.TIMER_THRESHOLD = timeout

    def get_move(self, game, legal_moves, time_left):
        """Search for the best move from the available legal moves and return a
        result before the time limit expires.

        This function must perform iterative deepening if self.iterative=True,
        and it must use the search method (minimax or alphabeta) corresponding
        to the self.method value.

        **********************************************************************
        NOTE: If time_left < 0 when this function returns, the agent will
              forfeit the game due to timeout. You must return _before_ the
              timer reaches 0.
        **********************************************************************

        Parameters
        ----------
        game : `isolation.Board`
            An instance of `isolation.Board` encoding the current state of the
            game (e.g., player locations and blocked cells).

        legal_moves : list<(int, int)>
            A list containing legal moves. Moves are encoded as tuples of pairs
            of ints defining the next (row, col) for the agent to occupy.

        time_left : callable
            A function that returns the number of milliseconds left in the
            current turn. Returning with any less than 0 ms remaining forfeits
            the game.

        Returns
        ----------
        (int, int)
            Board coordinates corresponding to a legal move; may return
            (-1, -1) if there are no available legal moves.
        """

        self.time_left = time_left
        # Perform any required initializations, including selecting an initial
        # move from the game board (i.e., an opening book), or returning
        # immediately if there are no legal moves
        potentialMoves = list()
        if len(legal_moves) == 0:
            return (-1, -1)

        try:
            # The search method call (alpha beta or minimax) should happen in
            # here in order to avoid timeout. The try/except block will
            # automatically catch the exception raised by the search method
            # when the timer gets close to expiring
            if self.search_depth < 0:
                self.search_depth = int(1e10)

            if self.iterative:
                if self.method == 'minimax':
                    for d in range(1, self.search_depth + 1):
                        potentialMoves.append(self.minimax(game, d))
                elif self.method == 'alphabeta':
                    for d in range(1, self.search_depth + 1):
                        potentialMoves.append(self.alphabeta(game, d))
            else:
                if self.method == 'minimax':
                    potentialMoves.append(self.minimax(game,
                                                       self.search_depth))
                elif self.method == 'alphabeta':
                    potentialMoves.append(self.alphabeta(game,
                                                         self.search_depth))

        except Timeout:
            pass

        # Return the best move from the last completed search iteration
        if potentialMoves != []:
            bestMove = max(potentialMoves)
            return bestMove[1]
        else:
            return (-1, -1)

    def minimax(self, game, depth, maximizing_player=True):
        """Implement the minimax search algorithm as described in the lectures.

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        maximizing_player : bool
            Flag indicating whether the current search depth corresponds to a
            maximizing layer (True) or a minimizing layer (False)

        Returns
        ----------
        float
            The score for the current search branch

        tuple(int, int)
            The best move for the current branch; (-1, -1) for no legal moves
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise Timeout()
        # initialize
        scores = []
        selectedMove = (-1, -1)
        # determine the player
        if maximizing_player:
            player = game.active_player
            scoreFinal = float("-inf")
        else:
            player = game.inactive_player
            scoreFinal = float("+inf")

        legal_moves = game.get_legal_moves()

        if len(legal_moves) == 0:
            score, selectedMove = self.score(game, player), (-1, -1)

        if depth == 1:
            for move in legal_moves:
                nextGame = game.forecast_move(move)
                score = self.score(nextGame, player)
                scores.append(score)
                if (maximizing_player and score > scoreFinal) or \
                   (not maximizing_player and score < scoreFinal):
                    scoreFinal, selectedMove = score, move
            return scoreFinal, selectedMove

        elif depth > 1:
            for move in legal_moves:
                nextGame = game.forecast_move(move)
                score = self.minimax(nextGame, depth - 1,
                                     not maximizing_player)[0]
                scores.append(score)
                if (maximizing_player and score > scoreFinal) or \
                   (not maximizing_player and score < scoreFinal):
                    scoreFinal, selectedMove = score, move
        return scoreFinal, selectedMove

    def alphabeta(self, game, depth, alpha=float("-inf"), beta=float("inf"),
                  maximizing_player=True):
        """Implement minimax search with alpha-beta pruning as described in the
        lectures.

        Parameters
        ----------
        game : isolation.Board
            An instance of the Isolation game `Board` class representing the
            current game state

        depth : int
            Depth is an integer representing the maximum number of plies to
            search in the game tree before aborting

        alpha : float
            Alpha limits the lower bound of search on minimizing layers

        beta : float
            Beta limits the upper bound of search on maximizing layers

        maximizing_player : bool
            Flag indicating whether the current search depth corresponds to a
            maximizing layer (True) or a minimizing layer (False)

        Returns
        ----------
        float
            The score for the current search branch

        tuple(int, int)
            The best move for the current branch; (-1, -1) for no legal moves
        """
        if self.time_left() < self.TIMER_THRESHOLD:
            raise Timeout()
        # initialize
        scores = []
        explore = True
        selectedMove = (-1, -1)
        # select player and initialize the resulting score
        if maximizing_player:
            player = game.active_player
            scoreFinal = float("-inf")
        else:
            player = game.inactive_player
            scoreFinal = float("+inf")
        # return the score and (-1, -1) if there are no legal moves
        legal_moves = game.get_legal_moves()
        if len(legal_moves) == 0:
            return (self.score(game, player), (-1, -1))
        # calculate the score for depth = 1
        if depth == 1:
            for move in legal_moves:
                if explore:
                    nextGame = game.forecast_move(move)
                    score = self.score(nextGame, player)
                    scores.append(score)
                    # update beta, the final score and the selected move
                    if not maximizing_player:
                        beta = min(beta, score)
                        if score < scoreFinal:
                            selectedMove = move
                            scoreFinal = score
                    # update alpha, the final score and the selected move
                    else:
                        alpha = max(alpha, score)
                        if score > scoreFinal:
                            selectedMove = move
                            scoreFinal = score
                    # interrupt the search if alpha >= beta
                    if alpha >= beta:
                        explore = False
            return scoreFinal, selectedMove
        # proceed with recursion with depth > 1
        elif depth > 1:
            for move in legal_moves:
                if explore:
                    nextGame = game.forecast_move(move)
                    score = self.alphabeta(nextGame, depth - 1,
                                           alpha,
                                           beta, not maximizing_player)[0]
                    scores.append(score)
                    # update beta, the final score and the selected move
                    if not maximizing_player:
                        beta = min(beta, score)
                        if score < scoreFinal:
                            selectedMove = move
                            scoreFinal = score
                    # update alpha, the final score and the selected move
                    else:
                        alpha = max(alpha, score)
                        if score > scoreFinal:
                            selectedMove = move
                            scoreFinal = score
                    # interrupt the search if alpha >= beta
                    if alpha >= beta:
                        explore = False
            # return result
            return scoreFinal, selectedMove

## Tournament

### Tournament

The `tournament.py` script is used to evaluate the effectiveness of your custom_score heuristic.  The script measures relative performance of your agent (called "Student") in a round-robin tournament against several other pre-defined agents.  The Student agent uses time-limited Iterative Deepening and the custom_score heuristic you wrote.

The performance of time-limited iterative deepening search is hardware dependent (faster hardware is expected to search deeper than slower hardware in the same amount of time).  The script controls for these effects by also measuring the baseline performance of an agent called "ID_Improved" that uess Iterative Deepening and the improved_score heuristic from `sample_players.py`.  Your goal is to develop a heuristic such that Student outperforms ID_Improved.

The tournament opponents are listed below. (See also: sample heuristics and players defined in sample_players.py)

- Random: An agent that randomly chooses a move each turn.
- MM_Null: CustomPlayer agent using fixed-depth minimax search and the null_score heuristic
- MM_Open: CustomPlayer agent using fixed-depth minimax search and the open_move_score heuristic
- MM_Improved: CustomPlayer agent using fixed-depth minimax search and the improved_score heuristic
- AB_Null: CustomPlayer agent using fixed-depth alpha-beta search and the null_score heuristic
- AB_Open: CustomPlayer agent using fixed-depth alpha-beta search and the open_move_score heuristic
- AB_Improved: CustomPlayer agent using fixed-depth alpha-beta search and the improved_score heuristic